# S&P 500 Data Analysis with XGBoost Model

We’re interested in building and refining a ML model that can classify a publicly traded company as presently overvalued or undervalued.  This would be based on Financial Statements of all companies trading on SP500 with data starting from 2010 through 2022. Companies can be grouped based on their valuation levels, then we can track if these groupings change over time or mostly stay consistent. 

We will optimize and compare models using different combinations of features of Financial Statements to find best logic for accurately valuing SP500 stocks. Here’s the dataset that will be used for this analysis:

https://www.kaggle.com/datasets/hanseopark/sp-500-stocks-value-with-financial-statement


### Common Valuation Formulas

This looks like it could be a classification problem. For reference. Four common metrics used for company valuation, we'll focus on Market Capitalization for starters.

- **Market Capitalization** = Share Price x Total Number of Shares
- Discounted Cash Flow = Terminal Cash Flow / (1 + Cost of Capital) # of Years in the Future
- Enterprise Value = Debt + Equity - Cash
- Value of a Growing Perpetuity = Cash Flow / (Cost of Capital - Growth Rate)

---

In [13]:
# Import modules and libraries
import os
import json
import pandas as pd
import numpy as np
import hvplot.pandas
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report
from pickle import dump, load
from pathlib import Path
from xgboost import XGBClassifier

import warnings

# Custom classes for interfacing with this dataset
from utils.financials import VBot
from utils.trades import VBotTrade

warnings.filterwarnings("ignore")

## Data Import

Pull in the data from our extensive cleansing process to review the first 3 rows.

In [14]:
# Read in data
data = Path("data/sp_500_merged_wsectors_data.csv")
df = pd.read_csv(data, delimiter=",").rename(columns={"Unnamed: 0":"Ticker"})
df = df.set_index("Ticker")
df.drop(columns=["0"], inplace=True) ### <=====

df.head(3)

,RecentPrice,Beta,AnnualDividendRate,ROE(%),ROA(%),ProfitMargin(%),TotalCash,TotalDebt,OutstandingShares,intangibleAssets,...,depreciation,otherCashflowsFromInvestingActivities,dividendsPaid,changeToAccountReceivables,changeToNetincome,capitalExpenditures,netBorrowings,otherCashflowsFromFinancingActivities,MarketCap,Sector
Ticker,,,,,,,,,,,,,,,,,,,,,
A,126.440002,1.05,0.79,24.200000,9.12,18.70,38600000.0,5.634000e+08,300110000.0,9.290000e+08,...,82000000.0,-1.000000e+06,-6.300000e+07,-46000000.0,1.230000e+08,-75000000.0,-3.774672e+08,1.247088e+09,3.794591e+10,Health Care
AAL,19.000000,1.59,0.00,57.579716,-5.04,-6.67,191500000.0,1.346056e+09,649160000.0,1.988000e+09,...,660000000.0,4.800000e+07,-4.160114e+08,-304000000.0,-1.969000e+09,-387000000.0,-6.100000e+08,5.400000e+07,1.233404e+10,NaN
AAP,218.679993,1.29,3.25,18.420000,4.69,5.60,99300000.0,1.226600e+09,61090000.0,6.512170e+08,...,65196000.0,1.078166e+08,-4.160114e+08,147953000.0,4.121900e+07,-98656000.0,-3.774672e+08,-7.000000e+03,1.335916e+10,NaN


In [15]:
df.shape

(152, 69)

## Split Build Fit and Train a RandomForestRegressor() Model


In [19]:
# Instantiate the model
xgb = XGBClassifier(n_estimators = 400, learning_rate = 0.1, max_depth = 3)

In [20]:
data = df.dropna()
y = data['MarketCap']
X = data.drop(columns='MarketCap')

In [21]:
# Encode the categorical variables using get_dummies
X = pd.get_dummies(X)

In [29]:
# Split dataset
seed = 7
test_size = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [30]:
display(y_test.head(3))
display(X_test.head(3))

Ticker
BA    1.069261e+10
Name: MarketCap, dtype: float64

,RecentPrice,Beta,AnnualDividendRate,ROE(%),ROA(%),ProfitMargin(%),TotalCash,TotalDebt,OutstandingShares,intangibleAssets,...,Sector_Consumer Discretionary,Sector_Consumer Staples,Sector_Energy,Sector_Financials,Sector_Health Care,Sector_Industrials,Sector_Information Technology,Sector_Materials,Sector_Telecommunications Services,Sector_Utilities
Ticker,,,,,,,,,,,,,,,,,,,,,
BA,181.940002,1.47,0.0,57.579716,-0.22,-6.75,278600000.0,1.346056e+09,58770000.0,2.562000e+09,...,0,0,0,0,0,1,0,0,0,0


In [31]:
X_test.shape

(1, 77)

In [32]:
# Scale the data
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [34]:
# Fit the data
le = LabelEncoder()
y_train = le.fit_transform(y_train)
xgb.fit(X_train.values, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=400,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=0, reg_alpha=0, ...)

In [36]:
# Create a list of feature values for prediction test
x_test_list = X_test.iloc[0].tolist()
X_test.iloc[0]#.index

RecentPrice                           181.940002
Beta                                    1.470000
AnnualDividendRate                      0.000000
ROE(%)                                 57.579716
ROA(%)                                 -0.220000
                                         ...    
Sector_Industrials                      1.000000
Sector_Information Technology           0.000000
Sector_Materials                        0.000000
Sector_Telecommunications Services      0.000000
Sector_Utilities                        0.000000
Name: BA, Length: 77, dtype: float64

In [38]:
# Make prediction dataframe using the test data
y_pred = xgb.predict(X_test_scaled)
ticlist = X_test.index.tolist()
results = pd.DataFrame({
    "Prediction": y_pred, 
    "Actual": y_test,
    "Ticker": ticlist
}).set_index("Ticker") #.reset_index(drop=True)

results.head(5)

,Prediction,Actual
Ticker,,
BA,0,1.069261e+10


In [39]:
results.hvplot(title="XG Boost Model on Market Capitalization - 37 Predictions", rot=60)

:NdOverlay   [Variable]
   :Curve   [Ticker]   (value)

In [40]:
# Train/Test Scores
reg = XGBClassifier(n_estimators = 500, learning_rate = 0.1, max_depth = 3, random_state=1).fit(X_train_scaled, y_train)
print(f'Train Score: {reg.score(X_train_scaled, y_train)}')
print(f'Test Score: {reg.score(X_test_scaled, y_test)}')

Train Score: 0.011363636363636364


ValueError: Classification metrics can't handle a mix of continuous and binary targets

In [ ]:
# Predict the MarketCap for (ANET) values
reg.predict(np.array([x_test_list]))

## Save the Model & Scaler files

In [17]:
# SAVE the model & scaler
dump(rfr, open('model/XGB_model.pkl', 'wb'))
dump(scaler, open('model/XGB_scaler.pkl', 'wb'))

## Load the Model & Scaler, then Predict!

In [18]:
# load the model & scaler
loaded_model = load(open('model/XGB_model.pkl', 'rb'))
loaded_scaler = load(open('model/XGB_scaler.pkl', 'rb'))

In [19]:
# Run the loaded scaler
# If this is run within this notebook, the results will be off. X_test has already been scaled a few cell above.
X_test_scaled = loaded_scaler.transform(X_test)

In [20]:
## Predict!
predictions = loaded_model.predict(X_test_scaled)
predictions

array([4.55678035e+10, 3.71534086e+11, 3.01370413e+10, 1.70548255e+11,
       2.48400369e+11, 1.73239760e+11, 2.69406253e+10, 2.70433259e+10,
       8.29088844e+10, 1.82990623e+10, 6.00030689e+10, 1.50467909e+11,
       2.75617035e+10, 1.58300649e+11, 2.80853872e+10, 1.18603796e+11,
       1.13059803e+11, 1.75907590e+10, 9.13526615e+10, 1.14896142e+11,
       3.31046706e+11, 2.69637864e+10, 3.12037385e+10])

In [21]:
predictions_df = pd.DataFrame(predictions)
predictions_df.hvplot()

:Curve   [index]   (0)